In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import psycopg2
import pandas as pd

In [4]:
DBNAME = "opportunity_youth"

In [5]:
conn = psycopg2.connect(dbname=DBNAME)
cur = conn.cursor()

In [6]:
### checking that all rows are for people, not houses
df = pd.read_sql("SELECT * FROM pums_2017;", conn)

,count
0,359075


In [7]:
### find table names from Database
cur.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cur.fetchall():
    print(table)

('pums_2017',)
('puma_names_2010',)
('wa_jobs_2017',)
('wa_geo_xwalk',)
('ct_puma_xwalk',)


In [36]:
### find King County puma id's from puma_names
df = pd.read_sql("""
SELECT puma, puma_name
FROM puma_names_2010
WHERE state_fips= '53' AND puma_name LIKE '%King%'
LIMIT 20
""",
           conn)

indexer = 0
for i in df['puma_name']:
    print (df['puma'][indexer] + ' : ' + i)
    indexer += 1

11606 : King County (Northwest)--Shoreline, Kenmore & Bothell (South) Cities                                                                                                                                                                                                                                                                                                                                                                                                                                                
11607 : King County (Northwest)--Redmond, Kirkland Cities, Inglewood & Finn Hill                                                                                                                                                                                                                                                                                                                                                                                                                           

In [39]:
### find seattle puma id's from puma
df = pd.read_sql("""
SELECT puma, puma_name
FROM puma_names_2010
WHERE puma_name LIKE '%Seattle%'
LIMIT 20
""",
           conn)

indexer = 0
for i in df['puma_name']:
    print (df['puma'][indexer] + ' : ' + i)
    indexer += 1

11601 : Seattle City (Northwest)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
11602 : Seattle City (Northeast)                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [10]:
### limiting data by South King County
pd.read_sql("""
SELECT COUNT(*)
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
""",
            conn)

,count
0,34145


In [11]:
### sql query that pulls all youth in SKC and writes to a csv file
ay_df = pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 16 AND 24
""",
            conn)

# ay_df.to_csv('./ay_df.csv')

the following csv files were moved to the Data folder after they were generated

In [12]:
### create 3 dataframes based on age grouping of ay_df

ay_df_16_18 =  pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 16 AND 18
""",
            conn)

# ay_df_16_18.to_csv('./ay_df_16_18.csv')

ay_df_19_21 =  pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 19 AND 21
""",
            conn)

# ay_df_19_21.to_csv('./ay_df_19_21.csv')

ay_df_22_24 =  pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 22 AND 24
""",
            conn)

# ay_df_22_24.to_csv('./ay_df_22_24.csv')

In [13]:
### limiting data by unemployed status & school status, assigning to oy_df dataframe and csv
oy_df = pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 16 AND 24
AND esr != '1'
AND esr != '4'
AND sch = '1'
""",
            conn)

oy_df

oy_total = sum(oy_df['pwgtp']) #used in racial calculations later
# oy_df.to_csv('./oy_df.csv')

the following csv files were moved to the Data folder after they were generated

In [14]:
### create 3 dataframes based on age grouping of oy_df and move to csv

oy_df_16_18 = pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 16 AND 18
AND esr != '1'
AND esr != '4'
AND sch = '1'
""",
            conn)

# oy_df_16_18.to_csv('./oy_df_16_18.csv')

oy_df_19_21 = pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 19 AND 21
AND esr != '1'
AND esr != '4'
AND sch = '1'
""",
            conn)

# oy_df_19_21.to_csv('./oy_df_19_21.csv')

oy_df_22_24 = pd.read_sql("""
SELECT *
FROM pums_2017
WHERE puma in ('11610', '11611', '11612', '11613', '11614', '11615')
AND agep BETWEEN 22 AND 24
AND esr != '1'
AND esr != '4'
AND sch = '1'
""",
            conn)

# oy_df_22_24.to_csv('./oy_df_22_24.csv')

In [15]:

print (sum(ay_df['pwgtp']))
print (sum(oy_df['pwgtp']))

overall_OY_rate = round((sum(oy_df['pwgtp'])) / (sum(ay_df['pwgtp'])) * 100)

overall_OY_rate
print (f'the rate of OY across the whole youth population is {overall_OY_rate}%')

85883.0
11115.0
the rate of OY across the whole youth population is 13%


In [16]:
white_ay = (sum(ay_df[ay_df['rac1p'] == '1']['pwgtp']))
white_oy = (sum(oy_df[oy_df['rac1p'] == '1']['pwgtp']))

white_oy_rate = round((white_oy / white_ay * 100))
white_oy_rate

white_prop_oy = round((white_oy / oy_total*100))
print (white_oy_rate)
print (white_prop_oy)

12
50


In [17]:
african_american_ay = (sum(ay_df[ay_df['rac1p'] == '2']['pwgtp']))
african_american_oy = (sum(oy_df[oy_df['rac1p'] == '2']['pwgtp']))

african_american_oy_rate = round((african_american_oy / african_american_ay * 100))
african_american_oy_rate

african_american_prop_oy = round((african_american_oy / oy_total*100))

In [18]:
native_american_ay = sum(ay_df[ay_df['rac1p'] == '3']['pwgtp']) + sum(ay_df[ay_df['rac1p'] == '4']['pwgtp']) + sum(ay_df[ay_df['rac1p'] == '5']['pwgtp'])
native_american_oy = sum(oy_df[oy_df['rac1p'] == '3']['pwgtp']) + sum(oy_df[oy_df['rac1p'] == '4']['pwgtp']) + sum(oy_df[oy_df['rac1p'] == '5']['pwgtp'])

native_american_oy_rate = round((native_american_oy / native_american_ay * 100))
native_american_oy_rate

native_american_prop_oy = round((native_american_oy / oy_total*100))

In [19]:
asian_ay = (sum(ay_df[ay_df['rac1p'] == '6']['pwgtp']))
asian_oy = (sum(oy_df[oy_df['rac1p'] == '6']['pwgtp']))

asian_oy_rate = round((asian_oy / asian_ay * 100))
asian_oy_rate

asian_prop_oy = round((asian_oy / oy_total*100))

In [20]:
pac_islander_ay = (sum(ay_df[ay_df['rac1p'] == '7']['pwgtp']))
pac_islander_oy = (sum(oy_df[oy_df['rac1p'] == '7']['pwgtp']))

pac_islander_oy_rate = round((pac_islander_oy / pac_islander_ay * 100))
pac_islander_oy_rate

pac_islander_prop_oy = round((pac_islander_oy / oy_total*100))

In [21]:
other_race_ay = (sum(ay_df[ay_df['rac1p'] == '8']['pwgtp']))
other_race_oy = (sum(oy_df[oy_df['rac1p'] == '8']['pwgtp']))

other_race_oy_rate = round((other_race_oy / other_race_ay * 100))
other_race_oy_rate

other_race_prop_oy = round((other_race_oy / oy_total*100))

In [22]:
multi_racial_ay = (sum(ay_df[ay_df['rac1p'] == '9']['pwgtp']))
multi_racial_oy = (sum(oy_df[oy_df['rac1p'] == '9']['pwgtp']))

multi_racial_oy_rate = round((multi_racial_oy / multi_racial_ay * 100))
multi_racial_oy_rate

multi_racial_prop_oy = round((multi_racial_oy / oy_total*100))

In [23]:
hispanic_ay = (sum(ay_df[ay_df['hisp'] != '01']['pwgtp']))
hispanic_oy = (sum(oy_df[oy_df['hisp'] != '01']['pwgtp']))

hispanic_oy_rate = round((hispanic_oy / hispanic_ay * 100))
hispanic_oy_rate

hispanic_prop_oy = round(((hispanic_oy / oy_total)*100))

#hispanic did not exist as a category in the rac1p, and we don't know what criterion the other reports used
#so we calculated our hispanic pop based on hisp = '01' i.e. Hispanic
#this resulted in a sum of propotions higher than 100%
#because we don't know how the groups were tabulated, we have no way of knowing if our data is even relevant 
#when comparing with previous studies
#which leads me to my next slide --- https://en.wikipedia.org/wiki/Radical_transparency

In [24]:
race = ['White', 'African American', 'Native American or Alaska Native', 'Pacific Islander', 'Asian', 'Other Race', 'Two or More Races', 'Hispanic']
ay_total_by_race = [white_ay, african_american_ay, native_american_ay, pac_islander_ay, asian_ay, other_race_ay, multi_racial_ay, hispanic_ay]
oy_total_by_race = [white_oy, african_american_oy, native_american_oy, pac_islander_oy, asian_oy, other_race_oy, multi_racial_oy, hispanic_oy]
racial_rate_of_oy = [white_oy_rate, african_american_oy_rate, native_american_oy_rate, pac_islander_oy_rate, asian_oy_rate, other_race_oy_rate, multi_racial_oy_rate, hispanic_oy_rate]
racial_proportion_of_oy = [white_prop_oy,african_american_prop_oy, native_american_prop_oy, pac_islander_prop_oy, asian_prop_oy, other_race_prop_oy, multi_racial_prop_oy, hispanic_prop_oy ]

racial_df = pd.DataFrame()

In [25]:
racial_df['Race'] = race
racial_df['Rate of OY'] = racial_rate_of_oy
racial_df['Total Youth Population'] = ay_total_by_race
racial_df['Proportion of OY'] = racial_proportion_of_oy
racial_df['OY Total'] = oy_total_by_race

In [26]:
# racial_df = racial_df.sort_values(by=['Rate of OY'], ascending=False)

In [27]:
racial_df

,Race,Rate of OY,Total Youth Population,Proportion of OY,OY Total
0,White,12,45663.0,50,5547.0
1,African American,15,8920.0,12,1343.0
2,Native American or Alaska Native,36,961.0,3,347.0
3,Pacific Islander,20,1877.0,3,373.0
4,Asian,9,13328.0,11,1196.0
5,Other Race,14,7298.0,9,1052.0
6,Two or More Races,16,7836.0,11,1257.0
7,Hispanic,14,15942.0,20,2239.0


In [28]:
# racial_df.to_csv('./racial_data.csv')

In [29]:
ls

01_erh_download_and_explore_data.ipynb  ay_df_22_24.csv
02_samt_exploratory.ipynb               oy_df.csv
Kyle_Exploration.ipynb                  oy_df_16_18.csv
README.md                               oy_df_19_21.csv
age_data.csv                            oy_df_22_24.csv
ay_df.csv                               project_outcomes.ipynb
ay_df_16_18.csv                         racial_data.csv
ay_df_19_21.csv


In [33]:
racial_df['Proportion of OY'].sum()

119

In [31]:
from src.example import *


In [32]:
say_hello()

hello
